In [ ]:
%cd /content
!git clone https://github.com/m-depascale/AnomalySegmentation_AML

/content
Cloning into 'AnomalySegmentation_AML'...
remote: Enumerating objects: 336, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 336 (delta 8), reused 13 (delta 8), pack-reused 320
Receiving objects: 100% (336/336), 185.59 MiB | 29.26 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [ ]:
!pip install ood-metrics
!pip install visdom
!pip install numpy
!pip install matplotlib
!pip install Pillow
!pip install torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408194 sha256=4c0032354172abdb23f2375f722a0216e876cae35e0bb85f82a7b18cc1d6c3d1
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


In [ ]:
!python -m pip install cityscapesscripts
!python -m pip install cityscapesscripts[gui]

In [ ]:
#As shows in the README here: https://github.com/mcordts/cityscapesScripts
from cityscapesscripts.download import downloader
print(dir(downloader))
import os

# Esegui il login
session = downloader.login()
names = downloader.list_available_packages(session=session)
print(names)


destination_path = "/content/Datasets/Cityscapes"
os.makedirs(destination_path, exist_ok=True)

downloader.download_packages(session=session, package_names=['leftImg8bit_trainvaltest.zip', 'gtFine_trainvaltest.zip'], destination_path=destination_path)


In [ ]:
!unzip /content/Datasets/Cityscapes/leftImg8bit_trainvaltest.zip -d /content/Datasets/Cityscapes/
!unzip /content/Datasets/Cityscapes/gtFine_trainvaltest.zip -d /content/Datasets/Cityscapes/

Streaming output truncated to the last 5000 lines.
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000117_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000114_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000434_000019_gtFine_labelIds.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000420_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000483_000019_gtFine_instanceIds.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000420_000019_gtFine_instanceIds.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000254_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000490_000019_gtFine_color.png  
  inflating: /content/Datasets/Cityscapes/gtFine/test/berlin/berlin_000448_000019_gtFine_polygons.json  
  inflating: /content

In [ ]:
os.environ['CITYSCAPES_DATASET'] = '/content/Datasets/Cityscapes'
from cityscapesscripts.preparation.createTrainIdLabelImgs import main as convertor
convertor()

Processing 5000 annotation files
Progress: 100.0 % 

# ERFNET TRAINING (PRUNED)

In [ ]:
%cd /content/AnomalySegmentation_AML/train

/content/AnomalySegmentation_AML/train


In [ ]:
# ERFNet full model definition for Pytorch
# Sept 2017
# Eduardo Romera
#######################

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

class DownsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()

        self.conv = nn.Conv2d(ninput, noutput-ninput, (3, 3), stride=2, padding=1, bias=True)
        self.pool = nn.MaxPool2d(2, stride=2)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = torch.cat([self.conv(input), self.pool(input)], 1)
        output = self.bn(output)
        return F.relu(output)


class non_bottleneck_1d (nn.Module):
    def __init__(self, chann, dropprob, dilated):
        super().__init__()

        self.conv3x1_1 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1,0), bias=True)

        self.conv1x3_1 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1), bias=True)

        self.bn1 = nn.BatchNorm2d(chann, eps=1e-03)

        self.conv3x1_2 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1*dilated,0), bias=True, dilation = (dilated,1))

        self.conv1x3_2 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1*dilated), bias=True, dilation = (1, dilated))

        self.bn2 = nn.BatchNorm2d(chann, eps=1e-03)

        self.dropout = nn.Dropout2d(dropprob)


    def forward(self, input):

        output = self.conv3x1_1(input)
        output = F.relu(output)
        output = self.conv1x3_1(output)
        output = self.bn1(output)
        output = F.relu(output)

        output = self.conv3x1_2(output)
        output = F.relu(output)
        output = self.conv1x3_2(output)
        output = self.bn2(output)

        if (self.dropout.p != 0):
            output = self.dropout(output)

        return F.relu(output+input)    #+input = identity (residual connection)


class Encoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.initial_block = DownsamplerBlock(3,16)

        self.layers = nn.ModuleList()

        self.layers.append(DownsamplerBlock(16,64))

        for x in range(0, 5):    #5 times
           self.layers.append(non_bottleneck_1d(64, 0.03, 1))

        self.layers.append(DownsamplerBlock(64,128))

        for x in range(0, 2):    #2 times
            self.layers.append(non_bottleneck_1d(128, 0.3, 2))
            self.layers.append(non_bottleneck_1d(128, 0.3, 4))
            self.layers.append(non_bottleneck_1d(128, 0.3, 8))
            self.layers.append(non_bottleneck_1d(128, 0.3, 16))

        #Only in encoder mode:
        self.output_conv = nn.Conv2d(128, num_classes, 1, stride=1, padding=0, bias=True)

    def forward(self, input, predict=False):
        output = self.initial_block(input)

        for layer in self.layers:
            output = layer(output)

        if predict:
            output = self.output_conv(output)

        return output


class UpsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()
        self.conv = nn.ConvTranspose2d(ninput, noutput, 3, stride=2, padding=1, output_padding=1, bias=True)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = self.conv(input)
        output = self.bn(output)
        return F.relu(output)

class Decoder (nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.layers = nn.ModuleList()

        self.layers.append(UpsamplerBlock(128,32))
        self.layers.append(non_bottleneck_1d(32, 0, 1))
        self.layers.append(non_bottleneck_1d(32, 0, 1))

        self.layers.append(UpsamplerBlock(32,16))
        self.layers.append(non_bottleneck_1d(16, 0, 1))
        self.layers.append(non_bottleneck_1d(16, 0, 1))

        self.output_conv = nn.ConvTranspose2d( 16, num_classes, 2, stride=2, padding=0, output_padding=0, bias=True)

    def forward(self, input):
        output = input

        for layer in self.layers:
            output = layer(output)

        output = self.output_conv(output)

        return output

#ERFNet
class ERFNET_PRUNED(nn.Module):
    def __init__(self, num_classes, encoder=None):  #use encoder to pass pretrained encoder
        super().__init__()

        if (encoder == None):
            self.encoder = Encoder(num_classes)
        else:
            self.encoder = encoder
        self.decoder = Decoder(num_classes)

    def forward(self, input, only_encode=False):
        if only_encode:
            return self.encoder.forward(input, predict=True)
        else:
            output = self.encoder(input)    #predict=False by default
            return self.decoder.forward(output)


In [ ]:
model = ERFNET_PRUNED(20)

In [ ]:
def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if 'decoder' in name:
                continue
            if name not in own_state:
                if name.startswith("module."):
                    own_state[name.split("module.")[-1]].copy_(param)
                else:
                    print(name, " not loaded")
                    continue
            else:
                own_state[name].copy_(param)
        return model

In [ ]:
model = load_my_state_dict(model, torch.load('/content/AnomalySegmentation_AML/trained_models/erfnet_pretrained.pth', map_location=lambda storage, loc: storage))


In [ ]:
for name, param in model.named_parameters():
   splits = name.split('.')
   if str(splits[0]) == 'encoder':
      param.requires_grad = False
   else:
      param.requires_grad = True

In [ ]:
%cd /content/AnomalySegmentation_AML/train

import os
import random
import time
import numpy as np
import torch
import math

from PIL import Image, ImageOps
from argparse import ArgumentParser

from torch.optim import SGD, Adam, lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, CenterCrop, Normalize, Resize, Pad
from torchvision.transforms import ToTensor, ToPILImage

from dataset import VOC12,cityscapes
from transform import Relabel, ToLabel, Colorize
from visualize import Dashboard

import importlib
from iouEval import iouEval, getColorEntry

from shutil import copyfile

NUM_CHANNELS = 3
NUM_CLASSES = 20 #pascal=22, cityscapes=20

color_transform = Colorize(NUM_CLASSES)
image_transform = ToPILImage()

#Augmentations - different function implemented to perform random augments on both image and target
class MyCoTransform(object):
    def __init__(self, enc, augment=True, height=512):
        self.enc=enc
        self.augment = augment
        self.height = height
        pass
    def __call__(self, input, target):
        # do something to both images
        input =  Resize(self.height, Image.BILINEAR)(input)
        target = Resize(self.height, Image.NEAREST)(target)

        if(self.augment):
            # Random hflip
            hflip = random.random()
            if (hflip < 0.5):
                input = input.transpose(Image.FLIP_LEFT_RIGHT)
                target = target.transpose(Image.FLIP_LEFT_RIGHT)

            #Random translation 0-2 pixels (fill rest with padding
            transX = random.randint(-2, 2)
            transY = random.randint(-2, 2)

            input = ImageOps.expand(input, border=(transX,transY,0,0), fill=0)
            target = ImageOps.expand(target, border=(transX,transY,0,0), fill=255) #pad label filling with 255
            input = input.crop((0, 0, input.size[0]-transX, input.size[1]-transY))
            target = target.crop((0, 0, target.size[0]-transX, target.size[1]-transY))

        input = ToTensor()(input)
        if (self.enc):
            target = Resize(int(self.height/8), Image.NEAREST)(target)
        target = ToLabel()(target)
        target = Relabel(255, 19)(target)

        return input, target


class CrossEntropyLoss2d(torch.nn.Module):

    def __init__(self, weight=None):
        super().__init__()

        self.loss = torch.nn.NLLLoss2d(weight)

    def forward(self, outputs, targets):
        return self.loss(torch.nn.functional.log_softmax(outputs, dim=1), targets)


def train(args, model, enc=False):
    best_acc = 0

    #TODO: calculate weights by processing dataset histogram (now its being set by hand from the torch values)
    #create a loder to run all images and calculate histogram of labels, then create weight array using class balancing

    weight = torch.ones(NUM_CLASSES)
    if (enc):
        weight[0] = 2.3653597831726
        weight[1] = 4.4237880706787
        weight[2] = 2.9691488742828
        weight[3] = 5.3442072868347
        weight[4] = 5.2983593940735
        weight[5] = 5.2275490760803
        weight[6] = 5.4394111633301
        weight[7] = 5.3659925460815
        weight[8] = 3.4170460700989
        weight[9] = 5.2414722442627
        weight[10] = 4.7376127243042
        weight[11] = 5.2286224365234
        weight[12] = 5.455126285553
        weight[13] = 4.3019247055054
        weight[14] = 5.4264230728149
        weight[15] = 5.4331531524658
        weight[16] = 5.433765411377
        weight[17] = 5.4631009101868
        weight[18] = 5.3947434425354
    else:
        weight[0] = 2.8149201869965
        weight[1] = 6.9850029945374
        weight[2] = 3.7890393733978
        weight[3] = 9.9428062438965
        weight[4] = 9.7702074050903
        weight[5] = 9.5110931396484
        weight[6] = 10.311357498169
        weight[7] = 10.026463508606
        weight[8] = 4.6323022842407
        weight[9] = 9.5608062744141
        weight[10] = 7.8698215484619
        weight[11] = 9.5168733596802
        weight[12] = 10.373730659485
        weight[13] = 6.6616044044495
        weight[14] = 10.260489463806
        weight[15] = 10.287888526917
        weight[16] = 10.289801597595
        weight[17] = 10.405355453491
        weight[18] = 10.138095855713

    weight[19] = 1.0

    assert os.path.exists(args['datadir']), "Error: datadir (dataset directory) could not be loaded"

    co_transform = MyCoTransform(enc, augment=True, height=args['height'])#1024)
    co_transform_val = MyCoTransform(enc, augment=False, height=args['height'])#1024)
    dataset_train = cityscapes(args['datadir'], co_transform, 'train')
    dataset_val = cityscapes(args['datadir'], co_transform_val, 'val')

    loader = DataLoader(dataset_train, num_workers=args['num_workers'], batch_size=args['batch_size'], shuffle=True)
    loader_val = DataLoader(dataset_val, num_workers=args['num_workers'], batch_size=args['batch_size'], shuffle=False)

    if args['cuda']:
        weight = weight.cuda()
    criterion = CrossEntropyLoss2d(weight)
    print(type(criterion))

    savedir = args['savedir']

    if (enc):
        automated_log_path = savedir + "/automated_log_encoder.txt"
        modeltxtpath = savedir + "/model_encoder.txt"
    else:
        automated_log_path = savedir + "/automated_log.txt"
        modeltxtpath = savedir + "/model.txt"

    if (not os.path.exists(automated_log_path)):    #dont add first line if it exists
        with open(automated_log_path, "a") as myfile:
            myfile.write("Epoch\t\tTrain-loss\t\tTest-loss\t\tTrain-IoU\t\tTest-IoU\t\tlearningRate")

    with open(modeltxtpath, "w") as myfile:
        myfile.write(str(model))

    optimizer = Adam(model.parameters(), 2e-4, (0.9, 0.999),  eps=1e-08, weight_decay=1e-4)

    start_epoch = 1
    if args['resume']:
        #Must load weights, optimizer, epoch and best value.
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'

        assert os.path.exists(filenameCheckpoint), "Error: resume option was used but checkpoint was not found in folder"
        checkpoint = torch.load(filenameCheckpoint)
        start_epoch = checkpoint['epoch']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['best_acc']
        print("=> Loaded checkpoint at epoch {})".format(checkpoint['epoch']))

    #scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5) # set up scheduler     ## scheduler 1
    lambda1 = lambda epoch: pow((1-((epoch-1)/args['num_epochs'])),0.9)  ## scheduler 2
    scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda1)                             ## scheduler 2

    if args['visualize'] and args['steps_plot'] > 0:
        board = Dashboard(8097)

    for epoch in range(start_epoch, args['num_epochs']+1):
        print("----- TRAINING - EPOCH", epoch, "-----")
        print(weight)
        scheduler.step(epoch)    ## scheduler 2

        epoch_loss = []
        time_train = []

        doIouTrain = args['iouTrain']
        doIouVal =  args['iouVal']

        if (doIouTrain):
            iouEvalTrain = iouEval(NUM_CLASSES)

        usedLr = 0
        for param_group in optimizer.param_groups:
            print("LEARNING RATE: ", param_group['lr'])
            usedLr = float(param_group['lr'])

        model.train()
        for step, (images, labels) in enumerate(loader):

            start_time = time.time()

            if args['cuda']:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images)
            targets = Variable(labels)
            outputs = model(inputs)
            #print(outputs.shape)


            optimizer.zero_grad()
            loss = criterion(outputs, targets[:, 0])
            loss.backward()
            optimizer.step()

            epoch_loss.append(loss.data.item())
            time_train.append(time.time() - start_time)

            if (doIouTrain):
                iouEvalTrain.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

            #print(outputs.size())
            if args['visualize'] and args['steps_plot'] > 0 and step % args['steps_plot'] == 0:
                start_time_plot = time.time()
                image = inputs[0].cpu().data

                board.image(image, f'input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'target (epoch: {epoch}, step: {step})')
                print ("Time to paint images: ", time.time() - start_time_plot)
            if args['steps_loss'] > 0 and step % args['steps_loss'] == 0:
                average = sum(epoch_loss) / len(epoch_loss)
                print(f'loss: {average:0.4} (epoch: {epoch}, step: {step})',
                        "// Avg time/img: %.4f s" % (sum(time_train) / len(time_train) / args['batch_size']))


        average_epoch_loss_train = sum(epoch_loss) / len(epoch_loss)

        iouTrain = 0
        if (doIouTrain):
            iouTrain, iou_classes = iouEvalTrain.getIoU()
            iouStr = getColorEntry(iouTrain)+'{:0.2f}'.format(iouTrain*100) + '\033[0m'
            print ("EPOCH IoU on TRAIN set: ", iouStr, "%")

        #Validate on 500 val images after each epoch of training
        print("----- VALIDATING - EPOCH", epoch, "-----")
        model.eval()
        epoch_loss_val = []
        time_val = []

        if (doIouVal):
            iouEvalVal = iouEval(NUM_CLASSES)

        for step, (images, labels) in enumerate(loader_val):
            start_time = time.time()
            if args['cuda']:
                images = images.cuda()
                labels = labels.cuda()

            inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
            targets = Variable(labels, volatile=True)
            outputs = model(inputs)

            loss = criterion(outputs, targets[:, 0])
            epoch_loss_val.append(loss.data.item())
            time_val.append(time.time() - start_time)


            #Add batch to calculate TP, FP and FN for iou estimation
            if (doIouVal):
                iouEvalVal.addBatch(outputs.max(1)[1].unsqueeze(1).data, targets.data)

            if args['visualize'] and args['steps_plot'] > 0 and step % args['steps_plot'] == 0:
                start_time_plot = time.time()
                image = inputs[0].cpu().data
                board.image(image, f'VAL input (epoch: {epoch}, step: {step})')
                if isinstance(outputs, list):   #merge gpu tensors
                    board.image(color_transform(outputs[0][0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'VAL output (epoch: {epoch}, step: {step})')
                else:
                    board.image(color_transform(outputs[0].cpu().max(0)[1].data.unsqueeze(0)),
                    f'VAL output (epoch: {epoch}, step: {step})')
                board.image(color_transform(targets[0].cpu().data),
                    f'VAL target (epoch: {epoch}, step: {step})')
                print ("Time to paint images: ", time.time() - start_time_plot)
            if args['steps_loss'] > 0 and step % args['steps_loss'] == 0:
                average = sum(epoch_loss_val) / len(epoch_loss_val)
                print(f'VAL loss: {average:0.4} (epoch: {epoch}, step: {step})',
                        "// Avg time/img: %.4f s" % (sum(time_val) / len(time_val) / args['batch_size']))


        average_epoch_loss_val = sum(epoch_loss_val) / len(epoch_loss_val)

        iouVal = 0
        if (doIouVal):
            iouVal, iou_classes = iouEvalVal.getIoU()
            iouStr = getColorEntry(iouVal)+'{:0.2f}'.format(iouVal*100) + '\033[0m'
            print ("EPOCH IoU on VAL set: ", iouStr, "%")


        if iouVal == 0:
            current_acc = -average_epoch_loss_val
        else:
            current_acc = iouVal
        is_best = current_acc > best_acc
        best_acc = max(current_acc, best_acc)
        if enc:
            filenameCheckpoint = savedir + '/checkpoint_enc.pth.tar'
            filenameBest = savedir + '/model_best_enc.pth.tar'
        else:
            filenameCheckpoint = savedir + '/checkpoint.pth.tar'
            filenameBest = savedir + '/model_best.pth.tar'
        save_checkpoint({
            'epoch': epoch + 1,
            'arch': str(model),
            'state_dict': model.state_dict(),
            'best_acc': best_acc,
            'optimizer' : optimizer.state_dict(),
        }, is_best, filenameCheckpoint, filenameBest)

        #SAVE MODEL AFTER EPOCH
        if (enc):
            filename = f'{savedir}/model_encoder-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_encoder_best.pth'
        else:
            filename = f'{savedir}/model-{epoch:03}.pth'
            filenamebest = f'{savedir}/model_best.pth'
        if args['epochs_save'] > 0 and step > 0 and step % args['epochs_save'] == 0:
            torch.save(model.state_dict(), filename)
            print(f'save: {filename} (epoch: {epoch})')
        if (is_best):
            torch.save(model.state_dict(), filenamebest)
            print(f'save: {filenamebest} (epoch: {epoch})')
            if (not enc):
                with open(savedir + "/best.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))
            else:
                with open(savedir + "/best_encoder.txt", "w") as myfile:
                    myfile.write("Best epoch is %d, with Val-IoU= %.4f" % (epoch, iouVal))

        #SAVE TO FILE A ROW WITH THE EPOCH RESULT (train loss, val loss, train IoU, val IoU)
        #Epoch		Train-loss		Test-loss	Train-IoU	Test-IoU		learningRate
        with open(automated_log_path, "a") as myfile:
            myfile.write("\n%d\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.4f\t\t%.8f" % (epoch, average_epoch_loss_train, average_epoch_loss_val, iouTrain, iouVal, usedLr ))

    return(model)   #return model (convenience for encoder-decoder training)

def save_checkpoint(state, is_best, filenameCheckpoint, filenameBest):
    torch.save(state, filenameCheckpoint)
    if is_best:
        print ("Saving model as best")
        torch.save(state, filenameBest)

/content/AnomalySegmentation_AML/train


In [ ]:
import logging
import logging.handlers
import sys
import os

savedir = f'../save/ERFNET_PRUNED'
if not os.path.exists(savedir):
        os.makedirs(savedir)

args = {
    'height': 512,
    'num_workers': 2,
    'batch_size' : 6,
    'cuda' : True,
    'savedir' : savedir,
    'resume': False,
    'num_epochs': 20,
    'steps_plot' : 50,
    'visualize' : False,
    'iouTrain': False,
    'iouVal': True,
    'steps_loss' : 50,
    'epochs_save': 0,
    'datadir': f'/content/Datasets/Cityscapes/',
    'resume': False
}

model.cuda()
erfnet_finetuned_20_classes = train(args, model)

/content/Datasets/Cityscapes/leftImg8bit/train
/content/Datasets/Cityscapes/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
tensor([ 2.8149,  6.9850,  3.7890,  9.9428,  9.7702,  9.5111, 10.3114, 10.0265,
         4.6323,  9.5608,  7.8698,  9.5169, 10.3737,  6.6616, 10.2605, 10.2879,
        10.2898, 10.4054, 10.1381,  1.0000], device='cuda:0')
LEARNING RATE:  0.0002
loss: 2.668 (epoch: 1, step: 0) // Avg time/img: 0.0640 s
loss: 2.458 (epoch: 1, step: 50) // Avg time/img: 0.0477 s
loss: 2.297 (epoch: 1, step: 100) // Avg time/img: 0.0477 s
loss: 2.147 (epoch: 1, step: 150) // Avg time/img: 0.0477 s
loss: 2.013 (epoch: 1, step: 200) // Avg time/img: 0.0478 s
loss: 1.891 (epoch: 1, step: 250) // Avg time/img: 0.0479 s
loss: 1.775 (epoch: 1, step: 300) // Avg time/img: 0.0478 s
loss: 1.68 (epoch: 1, step: 350) // Avg time/img: 0.0478 s
loss: 1.594 (epoch: 1, step: 400) // Avg time/img: 0.0478 s
loss: 1.52 (epoch: 1, step: 450) // Avg time/img: 0.0478 s

<ipython-input-26-1156a798cbcb>:277: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
<ipython-input-26-1156a798cbcb>:278: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = Variable(labels, volatile=True)


VAL loss: 0.642 (epoch: 1, step: 0) // Avg time/img: 0.0442 s
VAL loss: 0.9965 (epoch: 1, step: 50) // Avg time/img: 0.0330 s
EPOCH IoU on VAL set:  25.96 %
Saving model as best
save: ../save/ERFNET_PRUNED/model_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
tensor([ 2.8149,  6.9850,  3.7890,  9.9428,  9.7702,  9.5111, 10.3114, 10.0265,
         4.6323,  9.5608,  7.8698,  9.5169, 10.3737,  6.6616, 10.2605, 10.2879,
        10.2898, 10.4054, 10.1381,  1.0000], device='cuda:0')
LEARNING RATE:  0.00019097707632429996
loss: 0.8103 (epoch: 2, step: 0) // Avg time/img: 0.0604 s
loss: 0.8303 (epoch: 2, step: 50) // Avg time/img: 0.0481 s
loss: 0.8211 (epoch: 2, step: 100) // Avg time/img: 0.0480 s
loss: 0.7996 (epoch: 2, step: 150) // Avg time/img: 0.0479 s
loss: 0.7868 (epoch: 2, step: 200) // Avg time/img: 0.0478 s
loss: 0.7742 (epoch: 2, step: 250) // Avg time/img: 0.0479 s
loss: 0.7592 (epoch: 2, step: 300) // Avg time/img: 0.0479 s
loss: 0.7421 (epoch: 2, step: 350) // Avg time/img: 

In [ ]:
!cp /content/AnomalySegmentation_AML/save/ERFNET_PRUNED/model_best.pth /content/AnomalySegmentation_AML/trained_models/erfnet_pruned_decoder.pth

# ERFNET PRUNED ENCODER

In [ ]:
# ERFNet full model definition for Pytorch
# Sept 2017
# Eduardo Romera
#######################

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

class DownsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()

        self.conv = nn.Conv2d(ninput, noutput-ninput, (3, 3), stride=2, padding=1, bias=True)
        self.pool = nn.MaxPool2d(2, stride=2)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = torch.cat([self.conv(input), self.pool(input)], 1)
        output = self.bn(output)
        return F.relu(output)


class non_bottleneck_1d (nn.Module):
    def __init__(self, chann, dropprob, dilated, cout=0):
        super().__init__()

        self.conv3x1_1 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1,0), bias=True)

        self.conv1x3_1 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1), bias=True)

        self.bn1 = nn.BatchNorm2d(chann, eps=1e-03)

        self.conv3x1_2 = nn.Conv2d(chann, chann, (3, 1), stride=1, padding=(1*dilated,0), bias=True, dilation = (dilated,1))

        self.conv1x3_2 = nn.Conv2d(chann, chann, (1,3), stride=1, padding=(0,1*dilated), bias=True, dilation = (1, dilated))

        self.bn2 = nn.BatchNorm2d(chann, eps=1e-03)

        self.dropout = nn.Dropout2d(dropprob)


    def forward(self, input):

        output = self.conv3x1_1(input)
        output = F.relu(output)
        output = self.conv1x3_1(output)
        output = self.bn1(output)
        output = F.relu(output)

        output = self.conv3x1_2(output)
        output = F.relu(output)
        output = self.conv1x3_2(output)
        output = self.bn2(output)

        if (self.dropout.p != 0):
            output = self.dropout(output)

        return F.relu(output+input)    #+input = identity (residual connection)


class Encoder(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.initial_block = DownsamplerBlock(3,16)

        self.layers = nn.ModuleList()

        self.layers.append(DownsamplerBlock(16,64))

        for x in range(0, 5):    #5 times
           self.layers.append(non_bottleneck_1d(64, 0.03, 1))

        self.layers.append(DownsamplerBlock(64,128))

        #for x in range(0, 2):    #2 times
        self.layers.append(non_bottleneck_1d(128, 0.3, 2))
        self.layers.append(non_bottleneck_1d(128, 0.3, 4))
        self.layers.append(non_bottleneck_1d(128, 0.3, 8))
        self.layers.append(non_bottleneck_1d(128, 0.3, 16))

        self.layers.append(UpsamplerBlock(128,32))

        self.layers.append(non_bottleneck_1d(32, 0.15, 1))
        self.layers.append(non_bottleneck_1d(32, 0.15, 2))
        self.layers.append(non_bottleneck_1d(32, 0.15, 4))
        self.layers.append(non_bottleneck_1d(32, 0.15, 8))

        self.layers.append(DownsamplerBlock(32,128))
        #Only in encoder mode:
        self.output_conv = nn.Conv2d(128, num_classes, 1, stride=1, padding=0, bias=True)

    def forward(self, input, predict=False):
        output = self.initial_block(input)

        for layer in self.layers:
            output = layer(output)

        if predict:
            output = self.output_conv(output)

        return output


class UpsamplerBlock (nn.Module):
    def __init__(self, ninput, noutput):
        super().__init__()
        self.conv = nn.ConvTranspose2d(ninput, noutput, 3, stride=2, padding=1, output_padding=1, bias=True)
        self.bn = nn.BatchNorm2d(noutput, eps=1e-3)

    def forward(self, input):
        output = self.conv(input)
        output = self.bn(output)
        return F.relu(output)

class Decoder (nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.layers = nn.ModuleList()

        self.layers.append(UpsamplerBlock(128,32))
        self.layers.append(non_bottleneck_1d(32, 0, 1))
        self.layers.append(non_bottleneck_1d(32, 0, 1))

        self.layers.append(UpsamplerBlock(32,16))
        self.layers.append(non_bottleneck_1d(16, 0, 1))
        self.layers.append(non_bottleneck_1d(16, 0, 1))

        self.output_conv = nn.ConvTranspose2d( 16, num_classes, 2, stride=2, padding=0, output_padding=0, bias=True)

    def forward(self, input):
        output = input

        for layer in self.layers:
            output = layer(output)

        output = self.output_conv(output)

        return output

#ERFNet
class ERFNET_PRUNEDV2(nn.Module):
    def __init__(self, num_classes, encoder=None):  #use encoder to pass pretrained encoder
        super().__init__()

        if (encoder == None):
            self.encoder = Encoder(num_classes)
        else:
            self.encoder = encoder
        self.decoder = Decoder(num_classes)

    def forward(self, input, only_encode=False):
        if only_encode:
            return self.encoder.forward(input, predict=True)
        else:
            output = self.encoder(input)    #predict=False by default
            return self.decoder.forward(output)


In [ ]:
pru_enc = ERFNET_PRUNEDV2(20)
pru_enc.cuda()
summary(pru_enc, (3,224,224))

In [ ]:
def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if 'encoder.layers.11' in name:
                continue
            if 'encoder.layers.12' in name:
                continue
            if 'encoder.layers.13' in name:
                continue
            if 'encoder.layers.14' in name:
                continue
            if name not in own_state:
                if name.startswith("module."):
                    own_state[name.split("module.")[-1]].copy_(param)
                else:
                    print(name, " not loaded")
                    continue
            else:
                own_state[name].copy_(param)
        return model

In [ ]:
pruned2 = load_my_state_dict(pru_enc,  torch.load('/content/AnomalySegmentation_AML/trained_models/erfnet_pruned_decoder.pth', map_location=lambda storage, loc: storage))

In [ ]:
for name, param in pruned2.named_parameters():
   splits = name.split('.')
   if str(splits[0]) == 'encoder':
      param.requires_grad = True
   else:
      param.requires_grad = False

In [ ]:
import logging
import logging.handlers
import sys
import os

savedir = f'../save/ERFNET_PRUNED'
if not os.path.exists(savedir):
        os.makedirs(savedir)

args = {
    'height': 512,
    'num_workers': 2,
    'batch_size' : 8,
    'cuda' : True,
    'savedir' : savedir,
    'resume': False,
    'num_epochs': 10,
    'steps_plot' : 50,
    'visualize' : False,
    'iouTrain': False,
    'iouVal': True,
    'steps_loss' : 50,
    'epochs_save': 0,
    'datadir': f'/content/Datasets/Cityscapes/',
    'resume': False
}

In [ ]:
!cp /content/AnomalySegmentation_AML/save/ERFNET_PRUNED/model_best.pth /content/AnomalySegmentation_AML/trained_models/erfnet_pruned_all.pth

In [ ]:
net = ERFNET_PRUNEDV2(20)
def load_my_state_dict(model, state_dict):  #custom function to load model when not all dict elements
        own_state = model.state_dict()
        for name, param in state_dict.items():
            if name not in own_state:
                if name.startswith("module."):
                    own_state[name.split("module.")[-1]].copy_(param)
                else:
                    print(name, " not loaded")
                    continue
            else:
                own_state[name].copy_(param)
        return model
net = load_my_state_dict(pru_enc,  torch.load('/content/AnomalySegmentation_AML/trained_models/erfnet_pruned_all.pth', map_location=lambda storage, loc: storage))

In [ ]:
for name, param in net.named_parameters():
   splits = name.split('.')
   if str(splits[0]) == 'encoder':
      param.requires_grad = True
   else:
      param.requires_grad = True

In [ ]:
erfnet_pruned_all = train(args, net)

/content/Datasets/Cityscapes/leftImg8bit/train
/content/Datasets/Cityscapes/leftImg8bit/val
<class '__main__.CrossEntropyLoss2d'>
----- TRAINING - EPOCH 1 -----
tensor([ 2.8149,  6.9850,  3.7890,  9.9428,  9.7702,  9.5111, 10.3114, 10.0265,
         4.6323,  9.5608,  7.8698,  9.5169, 10.3737,  6.6616, 10.2605, 10.2879,
        10.2898, 10.4054, 10.1381,  1.0000], device='cuda:0')
LEARNING RATE:  0.0002


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:222: UserWarning: NLLLoss2d has been deprecated. Please use NLLLoss instead as a drop-in replacement and see https://pytorch.org/docs/master/nn.html#torch.nn.NLLLoss for more details.
  warnings.warn("NLLLoss2d has been deprecated. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:149: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is 

loss: 0.2366 (epoch: 1, step: 0) // Avg time/img: 0.1312 s
loss: 0.2877 (epoch: 1, step: 50) // Avg time/img: 0.1136 s
loss: 0.2845 (epoch: 1, step: 100) // Avg time/img: 0.1132 s
loss: 0.2862 (epoch: 1, step: 150) // Avg time/img: 0.1132 s
loss: 0.2882 (epoch: 1, step: 200) // Avg time/img: 0.1130 s
loss: 0.2889 (epoch: 1, step: 250) // Avg time/img: 0.1130 s
loss: 0.2925 (epoch: 1, step: 300) // Avg time/img: 0.1131 s
loss: 0.2962 (epoch: 1, step: 350) // Avg time/img: 0.1130 s
----- VALIDATING - EPOCH 1 -----


<ipython-input-26-1156a798cbcb>:277: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  inputs = Variable(images, volatile=True)    #volatile flag makes it free backward or outputs for eval
<ipython-input-26-1156a798cbcb>:278: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  targets = Variable(labels, volatile=True)


VAL loss: 0.2807 (epoch: 1, step: 0) // Avg time/img: 0.0359 s
VAL loss: 0.4179 (epoch: 1, step: 50) // Avg time/img: 0.0328 s
EPOCH IoU on VAL set:  59.32 %
Saving model as best
save: ../save/ERFNET_PRUNED/model_best.pth (epoch: 1)
----- TRAINING - EPOCH 2 -----
tensor([ 2.8149,  6.9850,  3.7890,  9.9428,  9.7702,  9.5111, 10.3114, 10.0265,
         4.6323,  9.5608,  7.8698,  9.5169, 10.3737,  6.6616, 10.2605, 10.2879,
        10.2898, 10.4054, 10.1381,  1.0000], device='cuda:0')
LEARNING RATE:  0.00018190651521659244
loss: 0.302 (epoch: 2, step: 0) // Avg time/img: 0.1226 s
loss: 0.2785 (epoch: 2, step: 50) // Avg time/img: 0.1128 s
loss: 0.2848 (epoch: 2, step: 100) // Avg time/img: 0.1130 s
loss: 0.2856 (epoch: 2, step: 150) // Avg time/img: 0.1128 s
loss: 0.2845 (epoch: 2, step: 200) // Avg time/img: 0.1128 s
loss: 0.2789 (epoch: 2, step: 250) // Avg time/img: 0.1129 s
loss: 0.2779 (epoch: 2, step: 300) // Avg time/img: 0.1129 s
loss: 0.2763 (epoch: 2, step: 350) // Avg time/img: 